### Transformer Based Encoding

In [ ]:
!pip install transformers==2.3.0

In [ ]:
# from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Loading the data

In [ ]:
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
import locale
locale.getpreferredencoding = getpreferredencoding

In [ ]:

path = '/content/drive/MyDrive/Capstone Data/Phase 2/final_data.pkl'
train_df = pd.read_pickle(path, compression = 'gzip')

features = ['sentence_count', 'word_count', 'unique_word_count', 
            'length', 'punctuation_count', 'upper_case_count', 
            'stopword_count', '#_count', 'unique_word_count_percent', 
            'Punctuation_percent', 'ip_count','link_count', 
            'article_id_count', 'username_count', 'clean_comment']

target_cols = ['toxic', 'severe_toxic', 'obscene', 'threat',
               'insult', 'identity_hate']
train_df = train_df.dropna()

# Seperating features and target variable
x = train_df[features]
y = train_df[target_cols]


In [ ]:
len(train_df[train_df['Sum']>0])

16225

#### Supporting Functions

In [ ]:
def evaluation_metrics(y_test, y_pred):
    result = {}
    result['Accuracy'] = accuracy_score(y_test, y_pred)
    result['Precision'] = precision_score(y_test, y_pred, average='weighted')
    result['Recall'] = recall_score(y_test, y_pred, average='weighted')
    result['F1 Score'] = f1_score(y_test, y_pred, average='weighted')
    return result

#### BERT Encoding

In [ ]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(x['clean_comment'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

  0%|          | 0/159571 [00:00<?, ?it/s]

In [ ]:
features.remove('clean_comment')

In [ ]:
merged_x = np.hstack((attention_masks, np.array(x[features])))
merged_x.shape

(159571, 142)

In [ ]:
# Splitting the data into train and test dataset
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(np.array(merged_x), np.array(y), 
                                            test_size = 0.2, random_state = 2)

In [ ]:
no_rows = train_x.shape[0]
no_cols = train_x.shape[1]

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

train_y_df = pd.DataFrame(train_y)
test_y_df = pd.DataFrame(test_y)
result_df = pd.DataFrame(['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])
result_list = []
for i in range(6):
  train_target = np.array(train_y_df[i]).reshape(-1,1)
  test_target = np.array(test_y_df[i]).reshape(-1,1)
  model = LogisticRegression(random_state=0).fit(train_x, train_target)
  pred = model.predict(test_x)
  pred = np.where(pred > 0.5, 1, 0)
  res = evaluation_metrics(test_target,pred)
  res = list(res.values())
  res.insert(0, target_cols[i])
  result_list.append(res) 

pd.DataFrame(result_list, columns=['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

,Target Variable,Accuracy,Precision,Recall,F1 Score
0,toxic,0.904308,0.883525,0.904308,0.861850
1,severe_toxic,0.989221,0.984576,0.989221,0.984193
2,obscene,0.947548,0.930614,0.947548,0.922909
3,threat,0.997149,0.994368,0.997149,0.995756
4,insult,0.951465,0.929876,0.951465,0.928532
5,identity_hate,0.991634,0.983338,0.991634,0.987469


#### Gaussian NB

In [ ]:
from sklearn.naive_bayes import GaussianNB


train_y_df = pd.DataFrame(train_y)
test_y_df = pd.DataFrame(test_y)
result_df = pd.DataFrame(['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])
result_list = []
for i in range(6):
  train_target = np.array(train_y_df[i]).reshape(-1,1)
  test_target = np.array(test_y_df[i]).reshape(-1,1)
  clf = GaussianNB()
  clf.fit(train_x, train_target)
  pred = clf.predict(test_x)
  pred = np.where(pred > 0.5, 1, 0)
  res = evaluation_metrics(test_target,pred)
  res = list(res.values())
  res.insert(0, target_cols[i])
  result_list.append(res) 

pd.DataFrame(result_list, columns=['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])



/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

,Target Variable,Accuracy,Precision,Recall,F1 Score
0,toxic,0.387467,0.865158,0.387467,0.473038
1,severe_toxic,0.515682,0.983842,0.515682,0.670096
2,obscene,0.369419,0.924221,0.369419,0.488038
3,threat,0.319348,0.995419,0.319348,0.480988
4,insult,0.346890,0.930242,0.346890,0.465881
5,identity_hate,0.338994,0.986233,0.338994,0.497455


#### Decision Tree

In [ ]:
from sklearn import tree

train_y_df = pd.DataFrame(train_y)
test_y_df = pd.DataFrame(test_y)
result_df = pd.DataFrame(['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])
result_list = []
for i in range(6):
  train_target = np.array(train_y_df[i]).reshape(-1,1)
  test_target = np.array(test_y_df[i]).reshape(-1,1)
  clf = tree.DecisionTreeClassifier()
  clf.fit(train_x, train_target)
  pred = clf.predict(test_x)
  pred = np.where(pred > 0.5, 1, 0)
  res = evaluation_metrics(test_target,pred)
  res = list(res.values())
  res.insert(0, target_cols[i])
  result_list.append(res) 

pd.DataFrame(result_list, columns=['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])



,Target Variable,Accuracy,Precision,Recall,F1 Score
0,toxic,0.841705,0.849620,0.841705,0.845563
1,severe_toxic,0.979571,0.980524,0.979571,0.980045
2,obscene,0.903901,0.911762,0.903901,0.907743
3,threat,0.993608,0.994522,0.993608,0.994063
4,insult,0.909384,0.916563,0.909384,0.912907
5,identity_hate,0.981545,0.983914,0.981545,0.982722


#### Sequential NN

In [ ]:
from keras.models import Sequential
from keras.layers import *
print(no_rows, no_cols)
nn_model = Sequential([
    Dense(512, input_shape=(no_cols,), activation='relu'),
    Dropout(0.2),
    # Dense(256, activation='relu'),
    # Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')

nn_model.summary()

127656 142
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               73216     
                                                                 
 dropout_28 (Dropout)        (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dropout_29 (Dropout)        (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 64)                16448     
                                                                 
 dropout_30 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 32)    

In [ ]:
train_y_df = pd.DataFrame(train_y)
test_y_df = pd.DataFrame(test_y)
result_df = pd.DataFrame(['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])
result_list = []
for i in range(6):
  train_target = np.array(train_y_df[i]).reshape(-1,1)
  test_target = np.array(test_y_df[i]).reshape(-1,1)
  history = nn_model.fit(train_x, train_target, epochs=10, 
                         batch_size=64, validation_data=(test_x, test_target))
  nn_pred = nn_model.predict(test_x)
  nn_pred = np.where(nn_pred > 0.5, 1, 0)
  res = evaluation_metrics(test_target,nn_pred)
  res = list(res.values())
  res.insert(0, target_cols[i])
  result_list.append(res) 

pd.DataFrame(result_list, columns=['Target Variable','Accuracy', 'Precision', 'Recall', 'F1 Score'])

Epoch 1/10
1995/1995 [==============================] - 13s 5ms/step - loss: 0.3532 - acc: 0.9021 - val_loss: 0.2940 - val_acc: 0.9068
Epoch 2/10
1995/1995 [==============================] - 9s 4ms/step - loss: 0.2966 - acc: 0.9075 - val_loss: 0.2917 - val_acc: 0.9066
Epoch 3/10
1995/1995 [==============================] - 11s 5ms/step - loss: 0.2933 - acc: 0.9076 - val_loss: 0.2905 - val_acc: 0.9067
Epoch 4/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.2887 - acc: 0.9076 - val_loss: 0.2866 - val_acc: 0.9074
Epoch 5/10
1995/1995 [==============================] - 11s 5ms/step - loss: 0.2864 - acc: 0.9080 - val_loss: 0.2869 - val_acc: 0.9075
Epoch 6/10
1995/1995 [==============================] - 9s 5ms/step - loss: 0.2844 - acc: 0.9082 - val_loss: 0.2942 - val_acc: 0.9081
Epoch 7/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.2830 - acc: 0.9085 - val_loss: 0.2887 - val_acc: 0.9063
Epoch 8/10
1995/1995 [==============================] - 1

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
1995/1995 [==============================] - 11s 5ms/step - loss: 0.1866 - acc: 0.9469 - val_loss: 0.1845 - val_acc: 0.9476
Epoch 2/10
1995/1995 [==============================] - 9s 4ms/step - loss: 0.1841 - acc: 0.9467 - val_loss: 0.1840 - val_acc: 0.9474
Epoch 3/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1836 - acc: 0.9467 - val_loss: 0.1838 - val_acc: 0.9476
Epoch 4/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1834 - acc: 0.9469 - val_loss: 0.1844 - val_acc: 0.9476
Epoch 5/10
1995/1995 [==============================] - 9s 4ms/step - loss: 0.1832 - acc: 0.9472 - val_loss: 0.1848 - val_acc: 0.9474
Epoch 6/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1835 - acc: 0.9470 - val_loss: 0.1835 - val_acc: 0.9476
Epoch 7/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1826 - acc: 0.9471 - val_loss: 0.1826 - val_acc: 0.9475
Epoch 8/10
1995/1995 [==============================] - 9

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
1995/1995 [==============================] - 9s 4ms/step - loss: 0.1867 - acc: 0.9503 - val_loss: 0.1762 - val_acc: 0.9516
Epoch 2/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1799 - acc: 0.9502 - val_loss: 0.1747 - val_acc: 0.9517
Epoch 3/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1778 - acc: 0.9503 - val_loss: 0.1787 - val_acc: 0.9516
Epoch 4/10
1995/1995 [==============================] - 9s 4ms/step - loss: 0.1771 - acc: 0.9501 - val_loss: 0.1762 - val_acc: 0.9515
Epoch 5/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1773 - acc: 0.9502 - val_loss: 0.1747 - val_acc: 0.9515
Epoch 6/10
1995/1995 [==============================] - 10s 5ms/step - loss: 0.1764 - acc: 0.9503 - val_loss: 0.1741 - val_acc: 0.9515
Epoch 7/10
1995/1995 [==============================] - 9s 5ms/step - loss: 0.1762 - acc: 0.9504 - val_loss: 0.1750 - val_acc: 0.9516
Epoch 8/10
1995/1995 [==============================] - 9s

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Target Variable,Accuracy,Precision,Recall,F1 Score
0,toxic,0.907849,0.887654,0.907849,0.875459
1,severe_toxic,0.989190,0.978497,0.989190,0.983814
2,obscene,0.947392,0.937018,0.947392,0.921922
3,threat,0.997180,0.994368,0.997180,0.995772
4,insult,0.951590,0.933252,0.951590,0.928896
5,identity_hate,0.991634,0.983338,0.991634,0.987469


#### LSTM 

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Input, Bidirectional
import tensorflow as tf
from keras.models import Model
from tensorflow.keras import backend as K

In [ ]:
def f1(y_true, y_pred):    
  def recall_m(y_true, y_pred):
      TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
      
      recall = TP / (Positives+K.epsilon())    
      return recall 
  
  
  def precision_m(y_true, y_pred):
      TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  
      precision = TP / (Pred_Positives+K.epsilon())
      return precision 
  
  precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
  
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def build_LSTM(data):
  # Define the LSTM model
  model = Sequential()
  model.add(LSTM(128, input_shape = (data.shape[1], 1), return_sequences = True))
  model.add(Dropout(0.2))
  model.add(LSTM(128, return_sequences = True))
  model.add(Dropout(0.2))
  model.add(LSTM(32))
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', 'Recall', 'Precision', f1])

  return model

In [ ]:
train_y_df = pd.DataFrame(train_y)
test_y_df = pd.DataFrame(test_y)
result_df = pd.DataFrame(['Target Variable','Loss', 'Accuracy', 'Recall', 'Precision', 'F1 Score'])
result_list = []
for i in range(6):
  train_target = np.array(train_y_df[i]).reshape(-1,1)
  test_target = np.array(test_y_df[i]).reshape(-1,1)
  LSTM_model = build_LSTM(train_x)
  LSTM_model.fit(train_x, train_target, epochs = 10, batch_size = 64)
  test_metrics = LSTM_model.evaluate(test_x, test_target, 
                                     batch_size = 64, verbose = 0)
  test_metrics.insert(0, target_cols[i])
  print(test_metrics)
  result_list.append(test_metrics) 

pd.DataFrame(result_list, columns=['Target Variable','Loss', 'Accuracy', 'Recall', 'Precision', 'F1 Score'])

Epoch 1/10
1995/1995 [==============================] - 44s 19ms/step - loss: 0.2982 - accuracy: 0.9060 - recall: 0.0568 - precision: 0.5873 - f1: 0.0858
Epoch 2/10
1995/1995 [==============================] - 40s 20ms/step - loss: 0.2876 - accuracy: 0.9088 - recall: 0.1084 - precision: 0.6330 - f1: 0.1629
Epoch 3/10
1995/1995 [==============================] - 39s 19ms/step - loss: 0.2852 - accuracy: 0.9091 - recall: 0.1167 - precision: 0.6343 - f1: 0.1733
Epoch 4/10
1995/1995 [==============================] - 38s 19ms/step - loss: 0.2836 - accuracy: 0.9092 - recall: 0.1200 - precision: 0.6329 - f1: 0.1775
Epoch 5/10
1995/1995 [==============================] - 38s 19ms/step - loss: 0.2821 - accuracy: 0.9094 - recall: 0.1278 - precision: 0.6280 - f1: 0.1900
Epoch 6/10
1995/1995 [==============================] - 38s 19ms/step - loss: 0.2809 - accuracy: 0.9095 - recall: 0.1258 - precision: 0.6356 - f1: 0.1860
Epoch 7/10
1995/1995 [==============================] - 38s 19ms/step - loss

,Target Variable,Loss,Accuracy,Recall,Precision,F1 Score
0,toxic,0.282901,0.908413,0.117114,0.651079,0.174653
1,severe_toxic,0.051315,0.989190,0.000000,0.000000,0.000000
2,obscene,0.186184,0.947517,0.014277,0.571429,0.020026
3,threat,0.019373,0.997180,0.000000,0.000000,0.000000
4,insult,0.177265,0.951527,0.010329,0.533333,0.011755
5,identity_hate,0.045945,0.991634,0.000000,0.000000,0.000000
